In [19]:
import json
from pathlib import Path
import requests
from bs4 import BeautifulSoup

import time

from tqdm.notebook import trange, tqdm

import pandas as pd
import numpy as np

In [6]:
## 공공 데이터 API 키 설정

apiKeyPath = './api-key/public-data-gov-key.json'
apiKey = ''

with open(apiKeyPath) as json_file:
    loaded = json.load(json_file)
    apiKey = loaded['key']

In [7]:
## 법정동 코드 (CP949) 를 UTF-8 CSV 로 저장하기


rawFilePath = './dataset/district_code_b_20200117.txt'
csvFilePath = './dataset/district_code_b_20200117.csv'

if Path(csvFilePath).is_file():
    print("가공된 법정동 코드 파일이 이미 존재합니다.")
else: 
    print("가공된 법정동 코드 파일이 존재하지 않습니다.")
    dfDistrictCodeRaw = pd.read_csv(
        rawFilePath, 
        sep = '\t',
        encoding = 'CP949')

    dfDistrictCodeConverted = dfDistrictCodeRaw.set_axis(
        ['district_code', 'district_name', 'district_available'], 
        axis=1, inplace=False)

    dfDistrictCodeConverted = dfDistrictCodeConverted.replace(
        {
            'district_available': {'존재': True, '폐지': False}
        }
    )
    
    ## district_parent_code 추가 후 reorder
    dfDistrictCodeConverted['district_parent_code'] = dfDistrictCodeConverted['district_code'].apply(lambda x: int(str(x)[:5]))
    dfDistrictCodeConverted = dfDistrictCodeConverted[[
        'district_parent_code',
        'district_code',
        'district_name',
        'district_available'
    ]]

    print(dfDistrictCodeConverted['district_available'].unique())

    dfDistrictCodeConverted.to_csv(csvFilePath, mode='w', index=False)

가공된 법정동 코드 파일이 이미 존재합니다.


In [8]:
## 법정동 코드 Pandas 로 읽기
dfDistrictCode = pd.read_csv('./dataset/district_code_b_20200117.csv')

In [9]:
## 법정구 코드 수
districtParentCodeCount = len(dfDistrictCode['district_parent_code'].unique())
print('법정구 코드 수 {}'.format(districtParentCodeCount))

법정구 코드 수 461


In [10]:
## 법정구 (법정동 상위) 코드를 찾아주는 함수 작성
def findDistrictCodeDf(query):
    df = dfDistrictCode \
        .query('district_name.str.contains("{}") and district_available == True'
        .format(targetDistrict))
    
    return df

targetDistrict = '도선동'
findDistrictCodeDf(targetDistrict)

,district_parent_code,district_code,district_name,district_available
289,11200,1120010400,서울특별시 성동구 도선동,True


In [11]:
dfDistrictCode['district_name'].str.split(' ')

0                          [서울특별시]
1                     [서울특별시, 종로구]
2                [서울특별시, 종로구, 청운동]
3                [서울특별시, 종로구, 신교동]
4                [서울특별시, 종로구, 궁정동]
                   ...            
46169    [제주특별자치도, 서귀포시, 표선면, 하천리]
46170    [제주특별자치도, 서귀포시, 표선면, 성읍리]
46171    [제주특별자치도, 서귀포시, 표선면, 가시리]
46172    [제주특별자치도, 서귀포시, 표선면, 세화리]
46173    [제주특별자치도, 서귀포시, 표선면, 토산리]
Name: district_name, Length: 46174, dtype: object

In [17]:
## 아파트 실거래가 데이터
def getAptTradeResponse(code, ymd, key):
    apiPrefixGetAptTrade = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"
    urlGetAptTrade = "{}?type=xml&LAWD_CD={}&DEAL_YMD={}&serviceKey={}".format(
        apiPrefixGetAptTrade, 
        code,
        ymd,
        key
    )

    responseApiTrade = requests.get(urlGetAptTrade, verify = False)
    return responseApiTrade.text

# XML 파싱 후 필터링
def parseAptTradeResponse(text):
    parsed = BeautifulSoup(text, 'lxml-xml')
    rawXmlItems = parsed.findAll("item")

    colNames = [
        '법정동',
        '지역코드',
        '아파트',
        '지번',
        '년',
        '월',
        '일',
        '건축년도',
        '전용면적',
        '층',
        '거래금액'
    ]

    dfRaw = pd.DataFrame(index = [], columns = colNames)

    # XML 내 item 을 반복하며 pandas df 로 변경
    for rawXmlItem in rawXmlItems: 
        row = dict.fromkeys(colNames)

        for col in colNames:       
            try :
                row[col] = rawXmlItem.find(col).text
            except :
                row[col] = np.nan

        dfTemp = pd.DataFrame(
            [row],
            columns = colNames)

        dfRaw = dfRaw.append(dfTemp)

    # 컬럼 타입 변경 또는 추가
    dfReturn = dfRaw
    dfReturn['거래일'] = pd.to_datetime(dfRaw['년'] + '-' + dfRaw['월'] + '-' + dfRaw['일'])
    dfReturn['거래금액'] = pd.to_numeric(dfRaw['거래금액'].str.replace(',', ''))

    dfReturn['지역코드'] = pd.to_numeric(dfRaw['지역코드'])
    dfReturn['년'] = pd.to_numeric(dfRaw['년'])
    dfReturn['월'] = pd.to_numeric(dfRaw['월'])
    dfReturn['일'] = pd.to_numeric(dfRaw['일'])
    
    dfReturn['법정동'] = dfRaw['법정동'].str.strip()
    dfReturn['아파트'] = dfRaw['아파트'].str.strip()
    dfReturn['지번'] = dfRaw['지번'].str.strip()

    dfReturn['건축년도'] = pd.to_numeric(dfRaw['건축년도'])
    dfReturn['전용면적'] = pd.to_numeric(dfRaw['전용면적'])
    dfReturn['층'] = pd.to_numeric(dfRaw['층'])
    
    dfReturn['p_ymd'] = pd.Timestamp("today").strftime("%Y%m%d")
    dfReturn.index = range(len(dfReturn))

    return dfReturn

In [14]:
def crawlAptTrade(code, ymd, key):

    
    filePath = './dataset/apt-trade/{}/apt-trade-{}-{}.csv'.format(ymd, code, ymd)
    
    resAptTrade = getAptTradeResponse(
        code, 
        ymd,
        key)

    dfAptTrade = parseAptTradeResponse(resAptTrade)
    
    if dfAptTrade.size > 0:
        dfAptTrade.to_csv(filePath, mode='w', index=False)

In [26]:
dealYmd = '201912'
uniqueCodes = dfDistrictCode['district_parent_code'].unique()

print('아파트 실거래가 수집 ({})'.format(dealYmd))
for code in tqdm(uniqueCodes):
    
    print('수집중: {}'.format(code))
    
    time.sleep(0.10)
    
    crawlAptTrade(code, dealYmd, apiKey)

아파트 실거래가 수집 (201912)


수집중: 11000
수집중: 11110
수집중: 11140
수집중: 11170
수집중: 11200

